## EnTK Modules

There are mandatory modules to use for describing workflow as EnTK provides a PST model; Pipeline, Stage, and Task. Simple workflow starts with `AppManager`, for example:
```python
   
   from radical import entk
   appman = entk.AppManager()
   p = entk.Pipeline()
   s = entk.Stage()
   t = entk.Task()
   t.executable = "echo Hello World"
   
   s.add_tasks(t)
   p.add_stages(s)
   appman.workflow = []
   appman.run()
```

Please refer to the API  for more details about Pipeline, Stage, and Task from here: https://radicalentk.readthedocs.io/en/latest/api/app_create.html

### Start with import EnTK

In [3]:
from radical import entk

### Preparing a job:  resource requests
Resource request to HPC machines consists in a number of CPUs/GPUs, computing expected duration, queue name and project name to charge. This is similar to a batch script in Slurm or IBM LSF. In addition, EnTK provides a task/cpu-gpu control to allocate a different number of CPUs/GPUs for simulation/analysis phase.

In [31]:
resource_config = {'ornl_summit':
                   {'label': 'ornl.summit',
                    'walltime': 30,
                    'cpus': 168,
                    'gpus': 0,
                    'queue': 'batch',
                    'access_schema': 'local',
                    'project':'CSC393'}}

workflow_config = { 'ornl_summit' : 
                   { 'simulation': 
                    { 'cpus': 168 },
                    'analysis': 
                    { 'cpus': 1 }}}

## Simple MDFF by EnTK

Let's try to integrate Simple MDFF with EnTK. According to the [Simple MDFF Tutorial](https://www.ks.uiuc.edu/Training/Tutorials/science/mdff/tutorial_mdff-html/), there are multiple steps:
- Get sample PDB files (Task 1)
- Generate a simulated density map (Task 1)
- Convert the density map to an MDFF potential (Task 2)
- Prepare the initial structure (Task 3)
- Defining secondary structure restraints (Task 4)
- Run the MDFF simulation with NAMD (Task 5, 6)
- Calculate the root mean square deviation (Task 7)

### Required command line tools
There are tools we will use in Simple MDFF:

- NAMD
- VMD

We use these on OLCF Summit:

In [ ]:
vmd_path = '/gpfs/alpine/world-shared/bip115/VMD_binaries/VMD-xlc-build-2019-Nov-11/bin/vmd'
namd_path = '/gpfs/alpine/world-shared/bip115/NAMD_binaries/summit/NAMD_build.latest/Linux-POWER-MPI-smp-Summit/namd2'

## VMD helper functions for tcl script

This function creates `tcl` script on the fly while Task is written in the following section.

In [4]:
def set_vmd_run(task, list_of_cmd, name=None):
    tcl_script_in_string = "\n".join(list_of_cmd) + "\nexit"
    task.pre_exec += [ "echo '{}' > {}".format(tcl_script_in_string, name or "input.tcl")]
    task.executable = vmd_path + ' -dispdev text -e ' + fname

### Start with Pipeline

In [5]:
p = Pipeline()
p.name = 'simple-mdff'


task1_output = ['4ake-target_autopsf.situs']
task2_output = ['4ake-target_autopsf-grid.dx']
task3_output = ['1ake-initial_autopsf-grid.pdb']
task4_output = ['1ake-extrabonds.txt', '1ake-extrabonds-cispeptide.txt', '1ake-extrabonds-chirality.txt']


first_stage = Stage()
# We use names of pipelines, stages, tasks to refer to data of a
# particular task
first_stage.name = 'Generating a simulated density map'


# Create tasks and add them to stage
task1 = Task()
task1.name = 'Starting to load the target PDB'
task1.cpu_reqs['threads_per_process'] = workflow_config['ornl_summit']['simulation']['cpus']
task1.pre_exec = [ "/usr/bin/wget https://www.ks.uiuc.edu/Training/Tutorials/science/mdff/mdff-tutorial-files/2-mdff-vacuo/4ake-target.pdb" ]
task1_tcl_cmds = [ 'mol new 4ake-target.pdb' ]
task1_tcl_cmds += [ 'package require autopsf' ]
task1_tcl_cmds += [ 'autopsf 4ake-target.pdb' ]
task1_tcl_cmds += [ 'set sel [atomselect top all]' ]
task1_tcl_cmds += [ 'package require mdff']
task1_tcl_cmds += [ 'mdff sim $sel -res 5 -o {}'.format(task1_output[0]) ]
task1_tcl_cmds += [ 'mol new {}'.format(task1_output[0]) ]

set_vmd_run(task1, task1_tcl_cmds, "first_stage.tcl")
first_stage.add_tasks(task1)
# Add sim_stage to Pipeline
p.add_stages(first_stage)


second_stage = Stage()
second_stage.name = 'Converting the density map to an MDFF potential'

task2 = Task()
task2.name = 'generate dx file'
task2.cpu_reqs['threads_per_process'] = workflow_config['ornl_summit']['simulation']['cpus']
task2_tcl_cmds = [ 'package require mdff' ]
task2_tcl_cmds += [ 'mdff griddx -i {} -o {}'.format(task1_output[0], task2_output[0]) ]
task2.copy_input_data = ['$Pipeline_{}_Stage_{}_Task_{}/{}'.format(p.name, first_stage.name, task1.name, task1_output[0])]


set_vmd_run(task2, task2_tcl_cmds, "second_stage.tcl")
second_stage.add_tasks(task2)
p.add_stages(second_stage)


third_stage = Stage()
third_stage.name = 'Preparing the initial structure'

task3 = Task()
task3.name = 'Starting to load the initial structure'
task3.cpu_reqs['threads_per_process'] = workflow_config['ornl_summit']['simulation']['cpus']
task3.pre_exec = [ '/usr/bin/wget https://www.ks.uiuc.edu/Training/Tutorials/science/mdff/mdff-tutorial-files/2-mdff-vacuo/1ake-initial.pdb' ]
task3_tcl_cmds = [ 'mol new 1ake-initial.pdb' ]
task3_tcl_cmds += [ 'package require autopsf' ]
task3_tcl_cmds += [ 'autopsf 1ake-initial.pdb' ]
task3_tcl_cmds += [ 'package require mdff' ]
task3_tcl_cmds += [ 'mdff gridpdb -psf 1ake-initial_autopsf.psf -pdb 1ake-initial_autopsf.pdb -o {}'.format(task3_output[0]) ]

set_vmd_run(task3, task3_tcl_cmds, "third_stage.tcl")
third_stage.add_tasks(task3)
p.add_stages(third_stage)


fourth_stage = Stage()
fourth_stage.name = 'Defining secondary structure restraints'

task4 = Task()
task4.cpu_reqs['threads_per_process'] = workflow_config['ornl_summit']['simulation']['cpus']
task4_tcl_cmds = [ 'package require ssrestraints' ]
task4_tcl_cmds += [ 'ssrestraints -psf 1ake-initial_autopsf.psf -pdb 1ake-initial_autopsf.pdb -o {} -hbonds'.format(task4_output[0]) ]
task4_tcl_cmds += [ 
        'mol new 1ake-initial_autopsf.psf',
        'mol addfile 1ake-initial_autopsf.pdb',
        'package require cispeptide',
        'package require chirality',
        'cispeptide restrain -o {}'.format(task4_output[1]),
        'chirality restrain -o {}'.format(task4_output[2]),
        ]
task4.copy_input_data = ['$Pipeline_{}_Stage_{}_Task_{}/{}'.format(p.name, third_stage.name, task3.name, '1ake-initial_autopsf.pdb'),
        '$Pipeline_{}_Stage_{}_Task_{}/{}'.format(p.name, third_stage.name, task3.name, '1ake-initial_autopsf.psf')]

set_vmd_run(task4, task4_tcl_cmds, "fourth_stage.tcl")
fourth_stage.add_tasks(task4)
p.add_stages(fourth_stage)


#fifth_stage = Stage()
#fifth_stage.name = 'Rigid-body docking the structure into the density map'
#task = Task()
#task.cpu_reqs['threads_per_process'] = workflow_config['ornl_summit']['simulation']['cpus']
# Expect to run Situs package
# colores 4ake-target_autopsf.situs 1ake-initial_autopsf.pdb -res 5 -nopowell
# However, this example uses prepared pdb files by wget
#task.pre_exec = [ '/usr/bin/wget https://www.ks.uiuc.edu/Training/Tutorials/science/mdff/mdff-tutorial-files/2-mdff-vacuo/1ake-colores.pdb' ]
#task.executable = [ 'mv' ]
#task.arguments  = [ '1ake-colores.pdb', '1ake-initial_autopsf-docked.pdb' ]

#fifth_stage.add_tasks(task)
#p.add_stages(fifth_stage)


sixth_stage = Stage()
sixth_stage.name = 'Running the MDFF simulation with NAMD'
task6 = Task()
task6.cpu_reqs['threads_per_process'] = workflow_config['ornl_summit']['simulation']['cpus']
task6.pre_exec = [ '/usr/bin/wget https://www.ks.uiuc.edu/Training/Tutorials/science/mdff/mdff-tutorial-files/2-mdff-vacuo/1ake-colores.pdb' ]
task6.pre_exec += [ 'mv 1ake-colores.pdb 1ake-initial_autopsf-docked.pdb' ] 
task6_tcl_cmds = [ 'package require mdff' ]
task6_tcl_cmds += [ 'mdff setup -o adk -psf 1ake-initial_autopsf.psf ' \
        + '-pdb 1ake-initial_autopsf-docked.pdb ' \
        + '-griddx 4ake-target_autopsf-grid.dx ' \
        + '-gridpdb 1ake-initial_autopsf-grid.pdb ' \
        + '-extrab {1ake-extrabonds.txt 1ake-extrabonds-cispeptide.txt 1ake-extrabonds-chirality.txt} ' \
        + '-gscale 0.3 -numsteps 50000' ]
task6_tcl_cmds += [ 'mdff setup -o adk -psf 1ake-initial_autopsf.psf ' \
        + '-pdb 1ake-initial_autopsf-docked.pdb ' \
        + '-griddx 4ake-target_autopsf-grid.dx ' \
        + '-gridpdb 1ake-initial_autopsf-grid.pdb ' \
        + '-extrab {1ake-extrabonds.txt 1ake-extrabonds-cispeptide.txt 1ake-extrabonds-chirality.txt} ' \
        + '-gscale 10 -minsteps 2000 -numsteps 0 -step 2' ]
task6.copy_input_data = [ '$Pipeline_{}_Stage_{}_Task_{}/{}'.format(p.name, second_stage.name, task2.name, '4ake-target_autopsf-grid.dx'),
        '$Pipeline_{}_Stage_{}_Task_{}/{}'.format(p.name, third_stage.name, task3.name, '1ake-initial_autopsf.pdb'),
        '$Pipeline_{}_Stage_{}_Task_{}/{}'.format(p.name, third_stage.name, task3.name, '1ake-initial_autopsf.psf'),
        '$Pipeline_{}_Stage_{}_Task_{}/{}'.format(p.name, third_stage.name, task3.name, '1ake-initial_autopsf-grid.pdb'),
        '$Pipeline_{}_Stage_{}_Task_{}/{}'.format(p.name, fourth_stage.name, task4.name, task4_output[0]),
        '$Pipeline_{}_Stage_{}_Task_{}/{}'.format(p.name, fourth_stage.name, task4.name, task4_output[1]),
        '$Pipeline_{}_Stage_{}_Task_{}/{}'.format(p.name, fourth_stage.name, task4.name, task4_output[2])]


set_vmd_run(task6, task6_tcl_cmds, "sixth_stage.tcl")
sixth_stage.add_tasks(task6)
p.add_stages(sixth_stage)

### NAMD Task
In this task, we can allocate many cores for parallel execution of NAMD simulation by `sim_cpus` defined by `workflow_config` earlier. Also, there is `download_output_data` special variable to download output file to your local directory. For example, `.dcd` file will be downloaded on your current directory.

In [ ]:
seventh_stage = Stage()
seventh_stage.name = "NAMD simulation"
task7 = Task()
task7.cpu_reqs['threads_per_process'] = workflow_config['ornl_summit']['simulation']['cpus']
task7.executable = [ namd_path ]
task7.arguments = ['+ppn', sim_cpus, 'adk-step1.namd']
task7.copy_input_data = [ '$Pipeline_{}_Stage_{}_Task_{}/{}'.format(p.name, sixth_stage.name, task6.name, 'adk-step1.namd'),
    '$Pipeline_{}_Stage_{}_Task_{}/{}'.format(p.name, sixth_stage.name, task6.name, 'adk-step2.namd'),
    '$Pipeline_{}_Stage_{}_Task_{}/{}'.format(p.name, sixth_stage.name, task6.name, '1ake-initial_autopsf.psf'),
    '$Pipeline_{}_Stage_{}_Task_{}/{}'.format(p.name, sixth_stage.name, task6.name, '1ake-initial_autopsf-docked.pdb'),
    '$Pipeline_{}_Stage_{}_Task_{}/{}'.format(p.name, sixth_stage.name, task6.name, '1ake-initial_autopsf-grid.pdb'),
    '$Pipeline_{}_Stage_{}_Task_{}/{}'.format(p.name, sixth_stage.name, task6.name, '4ake-target_autopsf-grid.dx'),
    '$Pipeline_{}_Stage_{}_Task_{}/{}'.format(p.name, sixth_stage.name, task6.name, '1ake-extrabonds-chirality.txt'),
    '$Pipeline_{}_Stage_{}_Task_{}/{}'.format(p.name, sixth_stage.name, task6.name, '1ake-extrabonds-cispeptide.txt'),
    '$Pipeline_{}_Stage_{}_Task_{}/{}'.format(p.name, sixth_stage.name, task6.name, '1ake-extrabonds.txt'),
    '$Pipeline_{}_Stage_{}_Task_{}/{}'.format(p.name, sixth_stage.name, task6.name, 'mdff_template.namd'),
    '$Pipeline_{}_Stage_{}_Task_{}/{}'.format(p.name, sixth_stage.name, task6.name, 'par_all27_prot_lipid_na.inp')
    ]
task7.download_output_data = ['adk-step1.dcd']
seventh_stage.add_tasks(task7)
#task7_2 = Task()
#task7_2.cpu_reqs['threads_per_process'] = workflow_config['ornl_summit']['simulation']['cpus']    
#task7_2.executable = [ 'namd2' ]
#task7_2.executable = [ 'namd2' ]
#task7_2.arguments = ['+ppn', sim_cpus, 'adk-step2.namd']
#seventh_stage.add_tasks(task7_2)
p.add_stages(seventh_stage)

### RMSD and correlation Tasks

In [ ]:
# Visualizing the MDFF trajectory
#
# mol new 4ake-target_autopsf.psf    
# mol addfile 4ake-target_autopsf.pdb  
# mol new 1ake-initial_autopsf.psf   
# mol addfile 1ake-initial_autopsf-docked.pdb  
# mol addfile adk-step1.dcd  
# mol addfile adk-step2.dcd


eighth_stage = Stage()
eighth_stage.name = 'Calculating the root mean square deviation'
task8 = Task()
task8.cpu_reqs['threads_per_process'] = workflow_config['ornl_summit']['simulation']['cpus']
task8_tcl_cmds = [ 
        'package require mdff',
        'mdff check -rmsd -refpdb 4ake-target_autopsf.pdb',
        'set selbbref [atomselect 0 "backbone"]',
        'set selbb [atomselect 1 "backbone"]',
        '$selbb frame 0',
        'measure rmsd $selbb $selbbref',
        '$selbb frame last',
        'measure rmsd $selbb $selbbref'
        ]

task8.copy_input_data = [ '$Pipeline_{}_Stage_{}_Task_{}/{}'.format(p.name, first_stage.name, task1.name, '4ake-target_autopsf.pdb')]
set_vmd_run(task8, task8_tcl_cmds, "eighth_stage.tcl")
eighth_stage.add_tasks(task8)
p.add_stages(eighth_stage)


ninth_stage = Stage()
ninth_stage.name = 'Calculating the cross-correlation coefficient'
task9 = Task()
task9.cpu_reqs['threads_per_process'] = workflow_config['ornl_summit']['simulation']['cpus']
task9_tcl_cmds = [ 
        'package require mdff',
        'mdff check -ccc -map 4ake-target_autopsf.situs -res 5',
        'set selallref [atomselect 0 "all"]',
        'set selall [atomselect 1 "all"]',
        '$selall frame 0',
        'mdff ccc $selall -i 4ake-target_autopsf.situs -res 5',
        '$selall frame last',
        'mdff ccc $selall -i 4ake-target_autopsf.situs -res 5']

task9.copy_input_data = [ 
    '$Pipeline_{}_Stage_{}_Task_{}/{}'.format(p.name, first_stage.name, task1.name, '4ake-target_autopsf.situs')]
set_vmd_run(task9, task9_tcl_cmds, "ninth_stage.tcl")
ninth_stage.add_tasks(task9)
p.add_stages(ninth_stage)

### EnTK AppManager with RabbitMQ
EnTK uses RabbmitMQ for the communication channel and here we define which host will be served.

In [25]:
comms = { 'rabbitmq':{
    'hostname': 'two.radical-project.org',
    'port': 33239
}

# Create Application Manager
appman = AppManager(hostname=comms['rabbitmq']['hostname'], 
                        port=comms['rabbitmq']['port'])

EnTK session: re.session.login3.hrlee.018218.0003
Creating AppManagerSetting up RabbitMQ system                                 ok
                                                                              ok


In [32]:
# Create a dictionary describe five mandatory keys:
# resource, walltime, cores, queue and access_schema
resource = "ornl_summit"
res_dict = {

    'resource': resource_config[resource]['label'],
    'walltime': resource_config[resource]['walltime'],
    'cpus': resource_config[resource]['cpus'],
    'queue': resource_config[resource]['queue'],
    'access_schema': resource_config[resource]['access_schema'],
    'project': resource_config[resource]['project']
}
# Assign resource manager to the Application Manager
appman.resource_desc = res_dict

### Run EnTK for Simple MDFF (WIP on Summit)
- This step starts with `resource_desc` for job description, and `run()` method triggers the job to start.

In [33]:
# Assign the workflow as a set or list of Pipelines to the Application Manager
appman.workflow = [p]

# Run the Application Manager
appman.run()

Validating and assigning resource manager                                     ok
Setting up RabbitMQ system                                                    ok


                                                                              ok
create pilot manager                                                          ok
submit 1 pilot(s)
        [ornl.summit:168]
                                                                              ok


All components created
Update: simple_mdff state: SCHEDULING
Update: simple_mdff.Generating a simulated density map state: SCHEDULING
Update: simple_mdff.Generating a simulated density map.Starting to load the target PDB state: SCHEDULING
Update: simple_mdff.Generating a simulated density map.Starting to load the target PDB state: SCHEDULED
Update: simple_mdff.Generating a simulated density map state: SCHEDULED


create unit manager

/ccs/home/hrlee/venv/rp/lib/python2.7/site-packages/pymongo/topology.py:155: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "


...

Update: simple_mdff.Calculating the root mean square deviation.task.0006 state: EXECUTED
Update: simple_mdff.Calculating the root mean square deviation.task.0006 state: FAILED
Update: simple_mdff.Calculating the root mean square deviation state: DONE
Update: simple_mdff.Calculating the cross-correlation coefficient state: SCHEDULING
Update: simple_mdff.Calculating the cross-correlation coefficient.task.0007 state: SCHEDULING
Update: simple_mdff.Calculating the cross-correlation coefficient.task.0007 state: SCHEDULED
Update: simple_mdff.Calculating the cross-correlation coefficient state: SCHEDULED
Update: simple_mdff.Calculating the cross-correlation coefficient.task.0007 state: SUBMITTING


submit 1 unit(s)
        .                                                                     ok


Update: simple_mdff.Calculating the cross-correlation coefficient.task.0007 state: EXECUTED
Update: simple_mdff.Calculating the cross-correlation coefficient.task.0007 state: FAILED
Update: simple_mdff.Calculating the cross-correlation coefficient state: DONE
Update: simple_mdff state: DONE


wait for 1 pilot(s)
        O[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0-[0\[0|[0/[0                                                                      ok
closing session re.session.login3.hrlee.018218.0003                            \
close pilot manager                                                            \
wait for 1 pilot(s)
     

All components terminated
